# 数据科学大作业

In [3]:
import numpy as np
import pandas as pd

np.random.seed(666)

## 1. 读取数据 & 数据清理

In [13]:
df = pd.read_csv('adult.csv', na_values=['NaN'])
df.columns = [
    "age", "work_class", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"
]
print(df.sample(5))

       age work_class  fnlwgt   education  education_num       marital_status  \
42891   24    Private  269799   Assoc-voc             11        Never-married   
3118    35          ?  169809   Bachelors             13   Married-civ-spouse   
42443   51    Private  257126        10th              6   Married-civ-spouse   
27124   72    Private  107814     Masters             14        Never-married   
15294   33    Private  205950     HS-grad              9        Never-married   

             occupation    relationship    race    sex  capital_gain  \
42891   Exec-managerial   Not-in-family   White   Male             0   
3118                  ?         Husband   White   Male             0   
42443      Craft-repair         Husband   White   Male             0   
27124    Prof-specialty   Not-in-family   White   Male          2329   
15294     Other-service       Own-child   White   Male             0   

       capital_loss  hours_per_week  native_country   income  
42891            